In [2]:
import numpy as np
from scipy.signal import hilbert #Envolvente compleja

import matplotlib.pyplot as plt
import Pred_asc as pasc
import PA_functions as pa
from pathlib import Path
import napari
from PIL import Image
import os
from napari.utils import nbscreenshot

C:\Users\alberto.vicente\.conda\envs\alberto\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
class RfAnalyze: 
    
    def __init__(self,data=[], save = True, fs = 40,axis = 2):
        
        data = data.astype('int32')
        media = np.mean(data)
        data = data - media
        
        self.data = data
        self.fs = fs
        self.hilbert = []
        self.saved = False
        self.viewer = None
        self.analytic()
    
    def analytic(self,save = True, visualzie = False):
        
        if save:
            self.saved = save
            self.hilbert = hilbert(self.data)
            return self.hilbert
        
        return hilbert(self.data)
            
    
    def envelope(self, plot = False):
        
        if self.saved:
            print("Using saved")
            
            data = self.hilbert
        
        else:
            
            data = self.analytic(False)
            
        result = np.abs(data)
        
        if plot:
            
            self.visualize(result)
        
        return result
    
    def inst_phase(self, plot = False):
        
        if self.saved:
            print("Using saved")
            
            data = self.hilbert
        
        else:
            
            data = self.analytic(False)
        
        def func(x): 
            return (np.angle(x))
        
        result = np.apply_along_axis(func,2,data)
        
        if plot:
            
            self.visualize(result)

        return result
    
    def inst_freq(self, plot = False):
        
        if self.saved:
            print("Using saved")
            
            data = self.hilbert
        
        else:
            
            data = self.analytic(False)
        
        def func(x):
            
            return (np.diff(np.unwrap(np.angle(x))) / (2.0*np.pi) * self.fs)
        
        result = np.apply_along_axis(func,2,data)
        
        if plot:
            
            self.visualize(result)
        
        return result
    
    def visualize(self,data,name = "Imagen"):
        
        if self.viewer == None:
        
            self.viewer = napari.view_image(pasc.axis_napari(data), name = name)
        
        else:
            
            self.viewer.add_image(pasc.axis_napari(data))
        

In [4]:
class RfAligner:
    
    def __init__(self):
        pass
        
    def align(rfsignal,alignment_ref,min_ref):
        
        np.apply_along_axis(self.align1D,2,rfsignal,alignment_ref,min_ref)
        
    def align1D(rfsignal,alignment_ref,min_ref):
        
        analytical = hilbert(rfsignal)
        envelope = np.abs(analytical)

        array1d = rfsignal
        align = np.zeros(array1d.shape) #array de ceros con la shape de la señal
        maxi = np.argmax(array1d)
        pad = maxi - alignment_ref #distancia a la referencia

        if maxi > 100:

            print(maxi)

        if (pad > 0):

            if np.argmax(envelope)>min_ref: #Si el pico del envelope esta mas a la derecha que el low limit
                end = len(array1d[pad:]) #Selecciona los valores de pad en adelante y cuenta la longitud
                align[:end] = array1d[pad:] #Deja un hueco de 0s con el tamaño de pad y rellena el resto con la señal
                return align
            else:
                return rfsignal

        elif(pad<0):

            start = len(array1d[:pad]) #Selecciona los valores hasta el pad y cuenta la longitud

            align[np.abs(pad):] = array1d[:start] #Deja un hueco de 0s con el tamaño de pad al principio y rellena el resto con la señal

            return align

        else:

            return rfsignal
        
    
    

In [6]:
folder_to_read = Path(r'C:\\Users\\alberto.vicente\\Documents\\Ultrasonidos\\libreria python\\20221013_Exp_pyUT\\Desarrollo libreria\\datos_prueba\\couponsMat4\\') 
folder_to_save = Path(r'C:\\Users\\alberto.vicente\\Documents\\Ultrasonidos\\libreria python')

# Name of the files
ruta_data8 = folder_to_read /   r'c4_align_env_vol_10MHz_lense.tif'


# # Assigning images to variables
data = ndt.napari_read_tiff(ruta_data8,folder=False)
data.shape

(241, 128, 1352)

# FrAligner

In [8]:
align = RfAligner()

In [ ]:
aligned = align.align(data,)

# FrAnalyze

In [4]:
analyze = RfAnalyze(data)

In [5]:
hilb = analyze.analytic()

In [6]:
env = analyze.envelope(True)

Using saved


napari.manifest -> 'napari' could not be imported: Could not find file 'builtins.yaml' in module 'napari'


In [7]:
phase = analyze.inst_phase(True)

Using saved


In [34]:
freq = analyze.inst_freq(True)

Using saved


In [13]:
analyze.visualize(env)

napari.manifest -> 'napari' could not be imported: Could not find file 'builtins.yaml' in module 'napari'


viewer = napari.view_image(pasc.axis_napari(freq), name='freq')

# RENDIMIENTO

In [8]:
import time

In [9]:
start_time = time.time()
analyze.envelope()
time.time() - start_time

Using saved


0.1728217601776123

In [10]:
start_time = time.time()
analyze.inst_phase()
time.time() - start_time

Using saved


0.5206804275512695

In [13]:
start_time = time.time()
analyze.inst_freq()
time.time() - start_time

Using saved


2.3713910579681396

In [32]:
start_time = time.time()
for i in range(20):
    analyze.envelope()
(time.time() - start_time)/20

Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved


0.2630586624145508

In [33]:
start_time = time.time()
for i in range(20):
    analyze.inst_phase()
(time.time() - start_time)/20

Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved


0.7564560055732727

In [14]:
start_time = time.time()
for i in range(20):
    analyze.inst_freq()
(time.time() - start_time)/20

Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved
Using saved


2.348786211013794

In [17]:
start_time = time.time()
for i in range(20):
    get_env(data)
(time.time() - start_time)/20

1.1470179677009582

In [18]:
start_time = time.time()
for i in range(20):
    analytic = hilbert(data)
    np.apply_along_axis(get_inst_phase,2,analytic)
(time.time() - start_time)/20

1.6436349987983703

In [15]:
start_time = time.time()
for i in range(20):
    analytic = hilbert(data)
    np.apply_along_axis(get_inst_freq,2,analytic,40)
(time.time() - start_time)/20

NameError: name 'get_inst_freq' is not defined

In [12]:
napari.view_image(pasc.axis_napari(data), name =  "name")

napari.manifest -> 'napari' could not be imported: Could not find file 'builtins.yaml' in module 'napari'


Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 120.0, 63.5), zoom=3.504356846473029, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(676.0, 1.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=3, ndisplay=2, last_used=0, range=((0.0, 1352.0, 1.0), (0.0, 241.0, 1.0), (0.0, 128.0, 1.0)), current_step=(676, 120, 64), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'name' at 0x23114aead00>], scale_bar=ScaleBar(visible=False, colored=False, color=array([1., 0., 1., 1.], dtype=float32), ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10.0, box=False, box_color=array([0. , 0. , 0. , 0.6], dtype=float32), unit=None), text_overlay=TextOverlay(visible=False, color=array([0.5, 0.5, 0.5, 1. ], dtype=float32), font_size=10.0, position=<TextOverlayPosition.TOP_